<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jaiio/03-modelos/05-ensamble%2Bsvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de votación (LR+SS3+TFIDF) + SVM

En esta notebook se presetan los experimentos a partir de los datos etiquetados automáticamente a partir de las _features_ extraidas del train dataset con todas las estrategias + la construcción de un clasificador con SVM.

Para ello vamos a preprocesar los correos y aplicar:
- Bag of words,
- Pesado binario/no binario,
- Máquina de vector soporte (SVM).


## 0. Configuración de la ejecución

A continuación se genera la configuración de las estrategias que integrarán el sistema de votación para la obtención de instancias:

In [1]:
# Posibilidades : lr, ss3, tfidf
# Combinaciones:
# lr-ss3
# lr-tfidf
# ss3-tfidf
# lr-ss3-tfidf

estrategias_feature_extraction = ['tfidf', 'ss3']
ACUMULTATIVO = False

# Para el sistema de votación queda fijo
CANTIDAD_INSTANCIAS = 200

Se definen algunas características fijas:

In [2]:
# El archivo de test y el train con etiquetado manual es siempre el mismo
TEST_FILE = 'correos-test-jaiio-20.csv'
TRAIN_FILE_MANUAL = 'correos-train-jaiio-80.csv'
atributos_df = ['consulta', 'dia_semana', 'semana_del_mes', 'mes', 'cuatrimestre',
                  'anio', 'hora_discretizada', 'dni_discretizado', 'legajo_discretizado',
                  'posee_legajo', 'posee_telefono', 'carrera_valor', 'proveedor_correo',
                  'cantidad_caracteres', 'proporcion_mayusculas', 'proporcion_letras',
                  'cantidad_tildes', 'cantidad_palabras', 'cantidad_palabras_cortas',
                  'proporcion_palabras_distintas', 'frecuencia_signos_puntuacion',
                  'cantidad_oraciones', 'utiliza_codigo_asignatura', 'score', 'clase_e0', 'clase_e1']

Y se generan los datos en función de la cantidad de estrategias a utilizar:

In [3]:
# Se genera el nombre del archivo de instancias a ejecutar en función de las estrategias
TRAIN_FILE_E0 = f'dataset-{estrategias_feature_extraction[0]}-200-prep.csv'
TRAIN_FILE_E1 = f'dataset-{estrategias_feature_extraction[1]}-200-prep.csv'
texto = f'Los dataset a utilizar son: \n\t{TRAIN_FILE_E0} \n\t{TRAIN_FILE_E1}'

# En caso que se realice el ensamble entre las 3 estrategias
if len(estrategias_feature_extraction)==3:

  TRAIN_FILE_E2 = f'dataset-{estrategias_feature_extraction[2]}-200-prep.csv'
  texto = texto + f'\n\t{TRAIN_FILE_E2}'
  
  # Se define el if para que ante reiteradas ejecuciones de la notebook no se vuelva a insertar
  if 'clase_e2' not in atributos_df:
    atributos_df.append('clase_e2')

print(texto)

Los dataset a utilizar son: 
	dataset-tfidf-200-prep.csv 
	dataset-ss3-200-prep.csv


## 1. Instalación y Carga de librerías y funciones útiles

### 1.1 Instalación de librerías

Se instalan las librerías que no están en el entorno de Google Colab:

In [4]:
# Se instala gensim que es el que tiene el modelo Word2Vec
!pip install requests
!pip install wget

### 1.2 Funciones útiles

Se cargan funciones útiles desde el repo https://github.com/jumafernandez/clasificacion_correos para la carga y balanceo del dataset.

In [5]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_dataset.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_dataset.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_dataset import get_clases, cargar_dataset, consolidar_df

También se carga la función para preprocesar el texto que se usó en los otros modelos desde el repo: https://github.com/jumafernandez/clasificacion_correos.

In [6]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_preprocesamiento.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_preprocesamiento.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_preprocesamiento import preprocesar_correos

### 1.3. Carga de datos

Cargo la librería warnings para no mostrar las advertencias, pandas para el manejo de df y os para verificar la existencia de los archivos en la carga de datos. Además, cargo en memoria la URL de base de los datasets y una lista de las etiquetas de las distintas clases:

In [7]:
import warnings
import pandas as pd
from os import path
warnings.filterwarnings("ignore")

# Constantes con los datos
DS_DIR = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/50jaiio/consolidados/'

# Defino las clases y la cantidad a utilizar
etiquetas = get_clases()
CANTIDAD_CLASES = len(etiquetas)

Se cargan los dataframes de las estrategias seleccionadas en memoria con el preprocesamiento de los datos:

In [8]:
# Chequeo sobre si los archivos están en el working directory
download_files = not(path.exists(TRAIN_FILE_E0))

df_e0, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_E0, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

# Se ejecuta el preprocesamiento de correos sobre el campo Consulta de train y test
df_e0['consulta'] = pd.Series(preprocesar_correos(df_e0['consulta']))
test_df['consulta'] = pd.Series(preprocesar_correos(test_df['consulta']))

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(df_e0.clase.unique()), df_e0.clase.unique()))

# Me quedo con las N cantidad de instancias con mayor score por clase
if CANTIDAD_INSTANCIAS<200:
  df_e0 = df_e0.sort_values(['clase','score'], ascending=False).groupby('clase').head(CANTIDAD_INSTANCIAS).reset_index(drop=True)
  df_e0 = df_e0.sample(frac = 1)


El conjunto de entrenamiento tiene la dimensión: (3200, 25)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 16 clases: ['Boleto Universitario' 'Cambio de Carrera' 'Cambio de Comisión'
 'Consulta por Equivalencias' 'Consulta por Legajo'
 'Consulta sobre Título Universitario' 'Cursadas' 'Datos Personales'
 'Exámenes' 'Ingreso a la Universidad' 'Pedido de Certificados'
 'Problemas con la Clave' 'Reincorporación' 'Requisitos de Ingreso'
 'Simultaneidad de Carreras' 'Situación Académica'].


In [9]:
# Chequeo sobre si los archivos están en el working directory
download_files = not(path.exists(TRAIN_FILE_E1))

df_e1, test_df_e1, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_E1, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

# Se ejecuta el preprocesamiento de correos sobre el campo Consulta de train y test
df_e1['consulta'] = pd.Series(preprocesar_correos(df_e1['consulta']))

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(df_e1.clase.unique()), df_e1.clase.unique()))

# Me quedo con las N cantidad de instancias con mayor score por clase
if CANTIDAD_INSTANCIAS<200:
  df_e1 = df_e1.sort_values(['clase','score'], ascending=False).groupby('clase').head(CANTIDAD_INSTANCIAS).reset_index(drop=True)
  df_e1 = df_e1.sample(frac = 1)


El conjunto de entrenamiento tiene la dimensión: (3200, 25)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 16 clases: ['Boleto Universitario' 'Cambio de Carrera' 'Cambio de Comisión'
 'Consulta por Equivalencias' 'Consulta por Legajo'
 'Consulta sobre Título Universitario' 'Cursadas' 'Datos Personales'
 'Exámenes' 'Ingreso a la Universidad' 'Pedido de Certificados'
 'Problemas con la Clave' 'Reincorporación' 'Requisitos de Ingreso'
 'Simultaneidad de Carreras' 'Situación Académica'].


Solo en el caso que decida hacer el sistema de votación entre las 3 estrategias inicializo la tercera:

In [10]:
if len(estrategias_feature_extraction)==3:

  # Chequeo sobre si los archivos están en el working directory
  download_files = not(path.exists(TRAIN_FILE_E2))

  df_e2, test_df_e2, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_E2, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

  # Se ejecuta el preprocesamiento de correos sobre el campo Consulta de train y test
  df_e2['consulta'] = pd.Series(preprocesar_correos(df_e2['consulta']))

  # Muestro salida por consola
  print('Existen {} clases: {}.'.format(len(df_e2.clase.unique()), df_e2.clase.unique()))

  # Me quedo con las N cantidad de instancias con mayor score por clase
  if CANTIDAD_INSTANCIAS<200:
    df_e2 = df_e2.sort_values(['clase','score'], ascending=False).groupby('clase').head(CANTIDAD_INSTANCIAS).reset_index(drop=True)
    df_e2 = df_e2.sample(frac = 1)

Verifico si voy a acumular los datos etiquetados manualmente a los de etiquetado no supervisado y en caso afirmativo los cargo en memoria:

In [11]:
if ACUMULTATIVO:
 
  # Chequeo sobre si los archivos están en el working directory
  download_files = not(path.exists(TRAIN_FILE_MANUAL))

  train_df_manual, test_df_manual, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_MANUAL, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

  # Se ejecuta el preprocesamiento de correos sobre el campo Consulta de train y test
  train_df_manual['consulta'] = pd.Series(preprocesar_correos(train_df_manual['consulta']))

  # Muestro salida por consola
  print('Existen {} clases: {}.'.format(len(train_df_manual.clase.unique()), train_df_manual.clase.unique()))

### 1.4 Sistema de votación entre estrategias de _feature extraction_

En primer lugar, se joinea mediante el texto de la consulta las estrategias encaradas (según sean dos o tres):

In [12]:
import pandas as pd

df_join = pd.merge(df_e0, df_e1, on='consulta', how='left', suffixes=(None, "_x"))

# Si están las 3 estrategias, se incorpora también la 3era
if len(estrategias_feature_extraction)==3:
  df_join = pd.merge(df_join, df_e2, on='consulta', how='left')

Borro las instancias con faltantes y verifico el resultado la dimensionalidad del df:

In [13]:
df_join = df_join.dropna()

df_join.shape

(843, 49)

In [14]:
df_join

,consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,score,clase,dia_semana_x,semana_del_mes_x,mes_x,cuatrimestre_x,anio_x,hora_discretizada_x,dni_discretizado_x,legajo_discretizado_x,posee_legajo_x,posee_telefono_x,carrera_valor_x,proveedor_correo_x,cantidad_caracteres_x,proporcion_mayusculas_x,proporcion_letras_x,cantidad_tildes_x,cantidad_palabras_x,cantidad_palabras_cortas_x,proporcion_palabras_distintas_x,frecuencia_signos_puntuacion_x,cantidad_oraciones_x,utiliza_codigo_asignatura_x,score_x,clase_x
0,buenas tardes quisiera saber posible cambiar h...,1,1,2,1,2019,1,5,0,0,0,3,33,218,0.0,0.788991,3,42,21,0.809524,0.022936,3,0,16.438839,Cambio de Carrera,1.0,1.0,2.0,1.0,2019.0,1.0,6.0,0.0,0.0,0.0,3.0,35.0,218.0,0.0,0.788991,3.0,42.0,21.0,0.809524,0.022936,3.0,0.0,18.420843,Cursadas
1,buenas tardes quisiera saber posible cambiar h...,1,1,2,1,2019,1,5,0,0,0,3,33,218,0.0,0.788991,3,42,21,0.809524,0.022936,3,0,16.438839,Cambio de Carrera,1.0,1.0,2.0,1.0,2019.0,1.0,6.0,0.0,0.0,0.0,3.0,35.0,218.0,0.0,0.788991,3.0,42.0,21.0,0.809524,0.022936,3.0,0.0,20.760212,Cambio de Comisión
2,olvide clave presione opcion restablecerla nun...,4,2,8,2,2018,3,4,2,1,1,3,13,148,0.0,0.810811,3,28,16,0.857143,0.006757,1,0,28.271154,Problemas con la Clave,4.0,2.0,8.0,2.0,2018.0,3.0,4.0,3.0,1.0,1.0,3.0,11.0,148.0,0.0,0.810811,3.0,28.0,16.0,0.857143,0.006757,1.0,0.0,33.542774,Problemas con la Clave
3,hola queria solicitar cambio horario asignatur...,3,2,2,1,2018,2,7,4,1,1,5,23,213,0.0,0.779343,2,37,18,0.810811,0.032864,4,0,18.615526,Datos Personales,3.0,2.0,2.0,1.0,2018.0,2.0,7.0,4.0,1.0,1.0,5.0,22.0,213.0,0.0,0.779343,2.0,37.0,18.0,0.810811,0.032864,4.0,0.0,19.202036,Cambio de Comisión
7,llego ningun correo urge reestablecer clave fa...,2,4,2,1,2018,3,8,4,1,1,41,13,114,0.0,0.815789,4,22,12,0.863636,0.000000,1,0,26.743103,Problemas con la Clave,2.0,4.0,2.0,1.0,2018.0,3.0,8.0,4.0,1.0,1.0,41.0,11.0,114.0,0.0,0.815789,4.0,22.0,12.0,0.863636,0.000000,1.0,0.0,32.358173,Problemas con la Clave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756,buenos dias pregunta acerca inscripcion aspira...,0,4,2,1,2018,0,8,0,0,0,0,23,217,0.0,0.811060,6,38,20,0.894737,0.018433,2,0,31.438173,Ingreso a la Universidad,0.0,4.0,2.0,1.0,2018.0,0.0,8.0,0.0,0.0,0.0,0.0,22.0,217.0,0.0,0.811060,6.0,38.0,20.0,0.894737,0.018433,2.0,0.0,28.790833,Ingreso a la Universidad
1757,buenos dias pregunta acerca inscripcion aspira...,0,4,2,1,2018,0,8,0,0,0,0,23,217,0.0,0.811060,6,38,20,0.894737,0.018433,2,0,31.438173,Ingreso a la Universidad,0.0,4.0,2.0,1.0,2018.0,0.0,8.0,0.0,0.0,0.0,0.0,22.0,217.0,0.0,0.811060,6.0,38.0,20.0,0.894737,0.018433,2.0,0.0,28.790833,Ingreso a la Universidad
1758,buenos dias pregunta acerca inscripcion aspira...,0,4,2,1,2018,0,8,0,0,0,0,23,217,0.0,0.811060,6,38,20,0.894737,0.018433,2,0,31.438173,Ingreso a la Universidad,0.0,4.0,2.0,1.0,2018.0,0.0,8.0,0.0,0.0,0.0,0.0,22.0,217.0,0.0,0.811060,6.0,38.0,20.0,0.894737,0.018433,2.0,0.0,28.790833,Ingreso a la Universidad
1760,queria solicitar certificado materias aprobada...,1,2,4,1,2017,3,5,2,1,1,43,23,70,0.0,0.842857,0,12,6,0.833333,0.000000,1,0,24.059593,Pedido de Certificados,1.0,2.0,4.0,1.0,2017.0,3.0,6.0,3.0,1.0,1.0,43.0,22.0,70.0,0.0,0.842857,0.0,12.0,6.0,0.833333,0.000000,1.0,0.0,29.226885,Pedido de Certificados


Muestro las primeras 5 instancias:

In [15]:
df_join.head()

,consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,score,clase,dia_semana_x,semana_del_mes_x,mes_x,cuatrimestre_x,anio_x,hora_discretizada_x,dni_discretizado_x,legajo_discretizado_x,posee_legajo_x,posee_telefono_x,carrera_valor_x,proveedor_correo_x,cantidad_caracteres_x,proporcion_mayusculas_x,proporcion_letras_x,cantidad_tildes_x,cantidad_palabras_x,cantidad_palabras_cortas_x,proporcion_palabras_distintas_x,frecuencia_signos_puntuacion_x,cantidad_oraciones_x,utiliza_codigo_asignatura_x,score_x,clase_x
0,buenas tardes quisiera saber posible cambiar h...,1,1,2,1,2019,1,5,0,0,0,3,33,218,0.0,0.788991,3,42,21,0.809524,0.022936,3,0,16.438839,Cambio de Carrera,1.0,1.0,2.0,1.0,2019.0,1.0,6.0,0.0,0.0,0.0,3.0,35.0,218.0,0.0,0.788991,3.0,42.0,21.0,0.809524,0.022936,3.0,0.0,18.420843,Cursadas
1,buenas tardes quisiera saber posible cambiar h...,1,1,2,1,2019,1,5,0,0,0,3,33,218,0.0,0.788991,3,42,21,0.809524,0.022936,3,0,16.438839,Cambio de Carrera,1.0,1.0,2.0,1.0,2019.0,1.0,6.0,0.0,0.0,0.0,3.0,35.0,218.0,0.0,0.788991,3.0,42.0,21.0,0.809524,0.022936,3.0,0.0,20.760212,Cambio de Comisión
2,olvide clave presione opcion restablecerla nun...,4,2,8,2,2018,3,4,2,1,1,3,13,148,0.0,0.810811,3,28,16,0.857143,0.006757,1,0,28.271154,Problemas con la Clave,4.0,2.0,8.0,2.0,2018.0,3.0,4.0,3.0,1.0,1.0,3.0,11.0,148.0,0.0,0.810811,3.0,28.0,16.0,0.857143,0.006757,1.0,0.0,33.542774,Problemas con la Clave
3,hola queria solicitar cambio horario asignatur...,3,2,2,1,2018,2,7,4,1,1,5,23,213,0.0,0.779343,2,37,18,0.810811,0.032864,4,0,18.615526,Datos Personales,3.0,2.0,2.0,1.0,2018.0,2.0,7.0,4.0,1.0,1.0,5.0,22.0,213.0,0.0,0.779343,2.0,37.0,18.0,0.810811,0.032864,4.0,0.0,19.202036,Cambio de Comisión
7,llego ningun correo urge reestablecer clave fa...,2,4,2,1,2018,3,8,4,1,1,41,13,114,0.0,0.815789,4,22,12,0.863636,0.000000,1,0,26.743103,Problemas con la Clave,2.0,4.0,2.0,1.0,2018.0,3.0,8.0,4.0,1.0,1.0,41.0,11.0,114.0,0.0,0.815789,4.0,22.0,12.0,0.863636,0.000000,1.0,0.0,32.358173,Problemas con la Clave


Renombro las clases en función de las estrategias y me quedo sólo con las instancias en que la clase coincide:

In [16]:
if len(estrategias_feature_extraction)==3:
  df_join.rename(columns={'clase_x': 'clase_e0', 'clase_y': 'clase_e1', 'clase': 'clase_e2'}, inplace=True)
  df_join['match_clase'] = ((df_join['clase_e0'] == df_join['clase_e1']) & (df_join['clase_e1'] == df_join['clase_e2']))
else:
  df_join.rename(columns={'clase': 'clase_e0', 'clase_x': 'clase_e1'}, inplace=True)
  df_join['match_clase'] = df_join['clase_e0'] == df_join['clase_e1']  
 
# Me quedo sólo con las instancias en que la clase coincide
train_df_join = df_join.query('match_clase == True').reset_index()

train_df_join.shape

(703, 51)

Me quedo solo con los atributos que me interesan (elimino duplicados):

In [17]:
train_df_join = train_df_join[atributos_df]

train_df_join.columns

Index(['consulta', 'dia_semana', 'semana_del_mes', 'mes', 'cuatrimestre',
       'anio', 'hora_discretizada', 'dni_discretizado', 'legajo_discretizado',
       'posee_legajo', 'posee_telefono', 'carrera_valor', 'proveedor_correo',
       'cantidad_caracteres', 'proporcion_mayusculas', 'proporcion_letras',
       'cantidad_tildes', 'cantidad_palabras', 'cantidad_palabras_cortas',
       'proporcion_palabras_distintas', 'frecuencia_signos_puntuacion',
       'cantidad_oraciones', 'utiliza_codigo_asignatura', 'score', 'clase_e0',
       'clase_e1'],
      dtype='object')

Me quedo sólo con una columna de clase dado que son las 3 iguales:

In [18]:
# Tomo una clase al azar, dado que coinciden las 3
train_df_join.rename(columns={'clase_e0': 'clase'}, inplace=True)

# Elimino las columnas que no necesito
train_df_join.drop(['clase_e1'], inplace=True, axis=1)
if len(estrategias_feature_extraction)==3:
  train_df_join.drop(['clase_e2'], inplace=True, axis=1)

train_df_join.shape

(703, 25)

In [19]:
train_df_join

,consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,score,clase
0,olvide clave presione opcion restablecerla nun...,4,2,8,2,2018,3,4,2,1,1,3,13,148,0.0,0.810811,3,28,16,0.857143,0.006757,1,0,28.271154,Problemas con la Clave
1,llego ningun correo urge reestablecer clave fa...,2,4,2,1,2018,3,8,4,1,1,41,13,114,0.0,0.815789,4,22,12,0.863636,0.000000,1,0,26.743103,Problemas con la Clave
2,cargan boleto estudiantil provincial quiero tr...,1,3,4,1,2018,3,1,2,1,1,4,40,207,0.0,0.801932,0,38,20,0.842105,0.019324,4,0,25.576574,Boleto Universitario
3,hola registro academico hora solicitar tramite...,0,4,3,1,2019,3,5,2,1,1,3,13,206,0.0,0.786408,3,37,21,0.891892,0.038835,3,0,19.925825,Consulta sobre Título Universitario
4,hola registro academico hora solicitar tramite...,0,4,3,1,2019,3,5,2,1,1,3,13,206,0.0,0.786408,3,37,21,0.891892,0.038835,3,0,19.925825,Consulta sobre Título Universitario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,buenos dias pregunta acerca inscripcion aspira...,0,4,2,1,2018,0,8,0,0,0,0,23,217,0.0,0.811060,6,38,20,0.894737,0.018433,2,0,31.438173,Ingreso a la Universidad
699,buenos dias pregunta acerca inscripcion aspira...,0,4,2,1,2018,0,8,0,0,0,0,23,217,0.0,0.811060,6,38,20,0.894737,0.018433,2,0,31.438173,Ingreso a la Universidad
700,buenos dias pregunta acerca inscripcion aspira...,0,4,2,1,2018,0,8,0,0,0,0,23,217,0.0,0.811060,6,38,20,0.894737,0.018433,2,0,31.438173,Ingreso a la Universidad
701,queria solicitar certificado materias aprobada...,1,2,4,1,2017,3,5,2,1,1,43,23,70,0.0,0.842857,0,12,6,0.833333,0.000000,1,0,24.059593,Pedido de Certificados


Me guardo la columna del score de Elasticsearch para no generar incompatibilidades:

In [20]:
score = train_df_join['score']
train_df_join.drop('score', inplace=True, axis=1)

In [21]:
if ACUMULTATIVO:
  train_df = pd.concat([train_df_manual, train_df_join], axis=0).reset_index(drop=True)
else:
  train_df = train_df_join

Muestro el dataframe resultante:

In [22]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
train_df.head()

,consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,clase
0,olvide clave presione opcion restablecerla nun...,4,2,8,2,2018,3,4,2,1,1,3,13,148,0.0,0.810811,3,28,16,0.857143,0.006757,1,0,Problemas con la Clave
1,llego ningun correo urge reestablecer clave fa...,2,4,2,1,2018,3,8,4,1,1,41,13,114,0.0,0.815789,4,22,12,0.863636,0.000000,1,0,Problemas con la Clave
2,cargan boleto estudiantil provincial quiero tr...,1,3,4,1,2018,3,1,2,1,1,4,40,207,0.0,0.801932,0,38,20,0.842105,0.019324,4,0,Boleto Universitario
3,hola registro academico hora solicitar tramite...,0,4,3,1,2019,3,5,2,1,1,3,13,206,0.0,0.786408,3,37,21,0.891892,0.038835,3,0,Consulta sobre Título Universitario
4,hola registro academico hora solicitar tramite...,0,4,3,1,2019,3,5,2,1,1,3,13,206,0.0,0.786408,3,37,21,0.891892,0.038835,3,0,Consulta sobre Título Universitario


## 2. SVM

### 2.1. Carga de librerías de procesamiento de texto

Se cargan en memoria dos funciones: _grid_search_por_estrategia_representacion_ que va a iterar ajustando los hiperparámetros para las técnica de __SVM__ y _representacion_documentos_ que genera representaciones para las _features textuales_:

In [23]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_clasificacion_texto.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_clasificacion_texto.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_clasificacion_texto import gridsearch_por_estrategia_representacion, representacion_documentos

### 2.2 Modelo general (clasificación en las 16 clases)

En primer lugar se trabaja con un único clasificador que clasifica y testea las instancias en las 16 clases posibles.

#### 2.2.1 Definición del espacio de búsqueda

Se define el espacio de búsqueda para el ajuste de hiperparámetros del modelo:

In [24]:
# Defino una lista con los esquemas de representación
estrategias_representacion = ['BINARIO', 'TFIDF', '3-4-NGRAM-CHARS', '1-2-NGRAM-WORDS']
modelo = 'SVM'

# Defino los parámetros para GridSearchCV
params_svm = {'SVM__C': [0.1, 1, 10, 100], 
              'SVM__gamma': [0.01, 0.1, 1],
              'SVM__class_weight': [None, 'balanced'],
              'SVM__kernel': ['rbf', 'linear', 'poly', 'sigmoid']
              }

Se ejecuta el ajuste de hiperparámetros para cada estrategia de representación en función del espacio de búsqueda:

In [ ]:
NO_CORRIDA = True

if NO_CORRIDA:
  for estrategia in estrategias_representacion:
    # Llamo a la función que realiza el gridsearch por estrategia  
    gridsearch_por_estrategia_representacion(train_df, test_df, estrategia, modelo, params_svm, None, atr_consulta='consulta')

Estrategia de representación: BINARIO
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  5.9min finished


Estrategia de representación: BINARIO
Parámetros: {'SVM__C': 10, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'clasificador': 'SVM', 'estrategia': 'BINARIO', 'accuracy': 0.545, 'precision': 0.8411167797888385, 'recall': 0.545, 'f1_score': 0.5999487408509705}
Accuracy Test-Set: 0.545
Métricas sobre Test-Set: {'SVM__C': 10, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'clasificador': 'SVM', 'estrategia': 'BINARIO', 'accuracy': 0.545, 'precision': 0.8411167797888385, 'recall': 0.545, 'f1_score': 0.5999487408509705}
Estrategia de representación: TFIDF
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  6.6min finished


Estrategia de representación: TFIDF
Parámetros: {'SVM__C': 100, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'clasificador': 'SVM', 'estrategia': 'TFIDF', 'accuracy': 0.565, 'precision': 0.7892167896435697, 'recall': 0.565, 'f1_score': 0.5957778148222911}
Accuracy Test-Set: 0.565
Métricas sobre Test-Set: {'SVM__C': 100, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'clasificador': 'SVM', 'estrategia': 'TFIDF', 'accuracy': 0.565, 'precision': 0.7892167896435697, 'recall': 0.565, 'f1_score': 0.5957778148222911}
Estrategia de representación: 3-4-NGRAM-CHARS
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 20.4min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 33.7min finished


Estrategia de representación: 3-4-NGRAM-CHARS
Parámetros: {'SVM__C': 1, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'clasificador': 'SVM', 'estrategia': '3-4-NGRAM-CHARS', 'accuracy': 0.58, 'precision': 0.8692989232989234, 'recall': 0.58, 'f1_score': 0.6272044653549238}
Accuracy Test-Set: 0.58
Métricas sobre Test-Set: {'SVM__C': 1, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'clasificador': 'SVM', 'estrategia': '3-4-NGRAM-CHARS', 'accuracy': 0.58, 'precision': 0.8692989232989234, 'recall': 0.58, 'f1_score': 0.6272044653549238}
Estrategia de representación: 1-2-NGRAM-WORDS
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  9.0min


#### 2.2.2 Modelo generado

En función de los mejores hiperparámetros encontrados con la búsqueda Grid, ajusto el modelo para obtener las métricas (por un bug en la función ya solucionado):

In [ ]:
# Mejores Parámetros según GridSearchCV:
# {'SVM__C': 10, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'estrategia': '3-4-NGRAM-CHARS', 
#  'accuracy': 0.81, 'precision': 0.5971321956813028, 'recall': 0.553190280233687, 'f1_score': 0.5639032258154839}
# Métricas sobre Test-Set: {'SVM__C': 10, 'SVM__class_weight': 'balanced', 'SVM__gamma': 0.01, 'SVM__kernel': 'sigmoid', 'SVM__probability': True, 'clasificador': 'SVM',
# 'estrategia': '3-4-NGRAM-CHARS', 'accuracy': 0.835, 'precision': 0.8428499293134182, 'recall': 0.835, 'f1_score': 0.8328323317634259}

params_svm = {'SVM__C': [1], 
              'SVM__gamma': [0.1],
              'SVM__class_weight': ['balanced'],
              'SVM__kernel': ['sigmoid'],
              'SVM__probability': [True]
              }
              
clf, X_test, y_test = gridsearch_por_estrategia_representacion(train_df, test_df, 'BINARIO', 'SVM', params_svm, None, atr_consulta='consulta')

from sklearn.metrics import matthews_corrcoef
y_pred = clf.predict(X_test)
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')

print(train_df.shape)

Se predicen las instancias de testeo:

In [ ]:
y_pred = clf.predict(X_test)

Además, se utiliza el método _predict\_proba_ para calcular la probabilidad asociada a la clasificación de cada instancia:

In [ ]:
import numpy as np

y_pred_proba = np.around(clf.predict_proba(X_test), 2)

y_max_proba = np.amax(y_pred_proba, 1)

In [ ]:
y_max_proba

Se genera un _dataframe_ con las clases observadas para _test_, las predicciones y las probabilidades asociadas para verificar si hay relación entre las proba y los errores:

In [ ]:
import pandas as pd

resultados = pd.concat([pd.Series(y_test), pd.Series(y_pred), pd.Series(y_max_proba)], axis=1)
resultados.columns = ['clase', 'prediccion', 'max_proba']
resultados['correcto'] = resultados['clase'] == resultados['prediccion']

Se calcula el _accuracy__ para las predicciones con una proba>0.5 y se obtiene una mejora sustancial:

In [ ]:
sum(resultados[resultados.max_proba>0.5]['correcto'])/resultados[resultados.max_proba>0.5]['correcto'].count()

#### 2.2.3 Métricas de selección

##### 2.2.3.1. Matriz de confusión

A continuación, se calcula la matriz de confusión para cada clase para el recorte con proba>0.5 y para el modelo con la totalidad de instancias:

In [ ]:
from sklearn import metrics #Importar el módulo metrics de scikit-learn

# Vamos a testear el modelo
print("Accuracy:", metrics.accuracy_score(resultados[resultados.max_proba>0.5]['clase'], resultados[resultados.max_proba>0.5]['prediccion']))

# Vemos un reporte de clasificación de varias métricas
print(metrics.classification_report(resultados[resultados.max_proba>0.5]['clase'], resultados[resultados.max_proba>0.5]['prediccion']))

In [ ]:
from sklearn import metrics #Importar el módulo metrics de scikit-learn

# Vamos a testear el modelo
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Vemos un reporte de clasificación de varias métricas
print(metrics.classification_report(y_test, y_pred))

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

mat = metrics.confusion_matrix(y_test, y_pred)

sns.heatmap(mat, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=etiquetas, yticklabels=etiquetas)

plt.xlabel('Observada')
plt.ylabel('Predicha');

### 3. Análisis del error

A continuación se intenta entender la baja de _accuracy_ para el etiquetado a partir de TF-IDF.

Para ello, se genera el dataset de train con: <br/>
| consulta | clase | score |

In [ ]:
df_train_e = train_df[['consulta', 'clase']]
df_train_e['score'] = score

Genero un dataframe con los scores promedios por clase, el accuracy por clase y el _count_:

In [ ]:
df_error = df_train_e.groupby(['clase']).mean().reset_index()

# Accuracy por clase
avg_class = mat.diagonal()/mat.sum(axis=1)
df_error['accuracy'] = pd.Series(avg_class)

df_error['count'] = mat.sum(axis=1)

df_error

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks")

sns.scatterplot(data=df_error, x="score", y="accuracy", hue="count", size="count")

## Referencias
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05